In [15]:
import pandas as pd
import sqlite3

## создаем подключение к базе данных с помощью библиотеки sqlite3

In [16]:
db = sqlite3.connect('../data/checking-logs.sqlite', check_same_thread=False)


## создать новую таблицу datamart в базе данных, объединив таблицы pageviews и checker, используя только один запрос

1. таблица должна иметь следующие столбцы: uid, labname, first_commit_ts, first_view_ts
2. first_commit_ts — это просто новое имя столбца timestamp из таблицы проверки, оно показывает первый коммит из конкретной лаборатории и от конкретного пользователя.
3. first_view_ts — первый визит пользователя в таблицу pageviews, временная метка, когда пользователь посетил ленту новостей
4. статус = «готов» все еще должен быть фильтром
5. numTrials = 1 все равно должен быть фильтром
6. имена лабораторий должны быть из списка: 'laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1'
7. таблица должна содержать только пользователей (uid с user_*), а не администраторов
8. first_commit_ts и first_view_ts следует анализировать как datetime64[ns]

In [17]:
query = """
SELECT checker.uid, 
    checker.labname,
    checker.timestamp AS first_commit_ts,
    pageviews.datetime AS first_view_ts
FROM checker 
    LEFT JOIN pageviews ON checker.uid=pageviews.uid
WHERE status='ready'
    AND numTrials=1
    AND labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
    AND checker.uid LIKE 'user_%'
GROUP BY
    checker.uid, checker.labname; 
"""
new_table = pd.io.sql.read_sql(query, db, parse_dates=['first_commit_ts', 'first_view_ts'])
new_table

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
...,...,...,...,...
135,user_8,laba04s,2020-04-19 10:22:35.761944,NaT
136,user_8,laba05,2020-05-02 13:28:07.705193,NaT
137,user_8,laba06,2020-05-16 17:56:15.755553,NaT
138,user_8,laba06s,2020-05-16 20:01:07.900727,NaT


## Используя методы Pandas, создайте два фрейма данных: тестовый и контрольный

1. тест должен иметь пользователей, которые имеют значения в first_view_ts


In [22]:
test = new_table[new_table['first_view_ts'].notnull()]
test

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
5,user_1,project1,2020-05-14 20:56:08.898880,2020-04-26 21:53:59.624136
6,user_10,laba04,2020-04-25 08:24:52.696624,2020-04-18 12:19:50.182714
7,user_10,laba04s,2020-04-25 08:37:54.604222,2020-04-18 12:19:50.182714
8,user_10,laba05,2020-05-01 19:27:26.063245,2020-04-18 12:19:50.182714
9,user_10,laba06,2020-05-19 11:39:28.885637,2020-04-18 12:19:50.182714


2. control должен иметь пользователей, у которых отсутствуют значения в first_view_ts


In [19]:
control = new_table[new_table['first_view_ts'].isnull()]
control

,uid,labname,first_commit_ts,first_view_ts
12,user_11,laba05,2020-05-03 21:06:55.970293,NaT
13,user_11,project1,2020-05-03 23:45:33.673409,NaT
14,user_12,laba04,2020-04-18 17:07:51.767358,NaT
15,user_12,laba04s,2020-04-26 15:42:38.070593,NaT
16,user_12,laba05,2020-05-03 08:39:25.174316,NaT
...,...,...,...,...
135,user_8,laba04s,2020-04-19 10:22:35.761944,NaT
136,user_8,laba05,2020-05-02 13:28:07.705193,NaT
137,user_8,laba06,2020-05-16 17:56:15.755553,NaT
138,user_8,laba06s,2020-05-16 20:01:07.900727,NaT


3. замените отсутствующие значения в control средним значением first_view_ts тестовых пользователей, мы будем использовать это значение для будущего анализа

In [20]:
control = control.fillna(test['first_view_ts'].mean())
control

,uid,labname,first_commit_ts,first_view_ts
12,user_11,laba05,2020-05-03 21:06:55.970293,2020-04-27 00:40:05.761783552
13,user_11,project1,2020-05-03 23:45:33.673409,2020-04-27 00:40:05.761783552
14,user_12,laba04,2020-04-18 17:07:51.767358,2020-04-27 00:40:05.761783552
15,user_12,laba04s,2020-04-26 15:42:38.070593,2020-04-27 00:40:05.761783552
16,user_12,laba05,2020-05-03 08:39:25.174316,2020-04-27 00:40:05.761783552
...,...,...,...,...
135,user_8,laba04s,2020-04-19 10:22:35.761944,2020-04-27 00:40:05.761783552
136,user_8,laba05,2020-05-02 13:28:07.705193,2020-04-27 00:40:05.761783552
137,user_8,laba06,2020-05-16 17:56:15.755553,2020-04-27 00:40:05.761783552
138,user_8,laba06s,2020-05-16 20:01:07.900727,2020-04-27 00:40:05.761783552


4. сохраните обе таблицы в базе данных, они вам понадобятся в следующих упражнениях.

In [21]:
test.to_sql('test', db)
control.to_sql('control', db)

ValueError: Table 'test' already exists.

In [ ]:
db.close()